This notebook implements a computation used in the proof of Lemma 7.2 of "Geometric decomposition of abelian varieties of order 1".

In [1]:
import sys
import time
from sage.all import *
from itertools import combinations

In [2]:
load("torsion_closure.sage")

In [3]:
t = time.time()

Compute the equation involving three roots of unity described in Lemma 5.1.

In [4]:
R.<alpha2, zeta1, zeta2, zeta3> = QQ[]
alpha1 = alpha2 * zeta3
a1 = alpha1 + 2 / alpha1 + 3
a2 = alpha2 + 2 / alpha2 + 3
e1 = a1 + a1^(- 1) - 4 - zeta1 - zeta1^(- 1)
e2 = a2 + a2^(- 1) - 4 - zeta2 - zeta2^(- 1)
u1 = e1.numerator().factor()[0][0]
u2 = e2.numerator().factor()[1][0]
f = u1.resultant(u2, alpha2)

Convert the answer into a Laurent polynomial, renormalizing to make the symmetry more visible. This confirms the statement of Lemma 5.1.

In [5]:
R1.<z1, z2, z3> = LaurentPolynomialRing(QQ, 3)
g = f(0, z1, z2, z3)/(-2*z1*z2*z3^2)
print(g)

z1*z2*z3^-1 + z1 + z2 + z3 - 2*z1*z2*z3^-2 - z1*z3^-1 - z2*z3^-1 - z2^-1*z3 - z1^-1*z3 - 2*z1^-1*z2^-1*z3^2 + z3^-1 + z2^-1 + z1^-1 + z1^-1*z2^-1*z3


Verify that the group $G$ preserves this polynomial.

In [6]:
assert g == g(z2, z1, z3)
assert g == g(~z1, ~z2, ~z3)
assert g == g(z1, ~z2, -z1/z3)

Compute the monomials of this Laurent polynomial, then remove the key terms $u = -z_1 z_2 z_3^{-2}$ and its inverse from the list.

In [7]:
mon = [i * g.monomial_coefficient(i).sign() for i in g.monomials()]
u = -z1*z2*z3^(-2)
mon.remove(u)
mon.remove(~u)

Generate candidates for the Laurent polynomial $h$, then compute $G$-orbits using a Cayley graph. Also include edges of the form $(h, g-h)$.

In [8]:
l2 = [sum(t)+u+~u for i in range(7) for t in combinations(mon, i) if all(~j in t for j in t)] + \
     [mu + s for mu in mon+[u,~u] for s in [-1, 1]]
edges = [(s, s(z2,z1,z3)) for s in l2] + [(s, s(~z1,~z2,~z3)) for s in l2] + [(s, s(z1, ~z2, -z1/z3)) for s in l2]
for s in l2:
    if len(list(s)) == 8:
        edges.append((s, g-s))
G = Graph(edges, loops=True) # This might add some new sums.
G1 = G.subgraph(vertices=l2) # Retain only the sums we started with.
l3 = [i[0] for i in G1.connected_components()]
print("{} orbits computed".format(len(l3)))

16 orbits computed


For each $h$, compute the torsion closure of the ideal $(g,h)$ in $\mathbb{Q}[z_1^{\pm}, z_2^{\pm}, z_3^{\pm}]$; this step is most of the runtime. To speed it up, we disable automatic cleanup of the results.

In [9]:
l4 = []
for h in l3:
    print(h)
    l4 += torsion_closure(R1.ideal([g, h]), raw=True)
print("Raw table complete")

z2 - z1*z2*z3^-2 - z2*z3^-1 - z2^-1*z3 - z1^-1*z2^-1*z3^2 + z2^-1


z3 - z1*z2*z3^-2 - z1*z3^-1 - z2*z3^-1 - z2^-1*z3 - z1^-1*z3 - z1^-1*z2^-1*z3^2 + z3^-1


Sort the resulting solutions into zero-dimensional and positive-dimensional ideals.

In [0]:
l5a = []
l5b = []
for I in l4:
    if I.dimension() > 0:
        l5a.append(I)
    else:
        l5b.append(I)

Remove duplicates from the positive-dimensional solutions, then print the results.

In [0]:
l5a = torsion_closure_canonicalize(l5a, QQ)
print(l5a)

Remove zero-dimensional solutions contained in a positive-dimensional solution.

In [0]:
l5b = [I for I in l5b if not any(J <= I for J in l5a)]

Sort the remaining zero-dimensional solutions by their base ring.

In [0]:
d = {}
for I in l5b:
    K = I.base_ring()
    if K not in d:
        d[K] = []
    d[K].append(I)
d2 = {}
for K in d:
    N = 2 if K == QQ else K.number_of_roots_of_unity()
    if N not in d2:
        d2[N] = []
    d2[N] += d[K]

Print the fields of definition of the remaining zero-dimensional solutions.

In [0]:
print(sorted(list(d2.keys())))

Remove duplicates and Galois conjugates among the zero-dimensional solutions. Currently this is not used for anything.

In [0]:
for N in d2:
    d2[N] = torsion_closure_canonicalize(d2[N], QQ)

Check for solutions involving roots of unity of different orders (proof of Corollary 1.2).

In [0]:
pairs = []
for N in [24, 30, 42]:
    K.<z> = CyclotomicField(N)
    RK.<z1K, z2K, z3K> = LaurentPolynomialRing(K, 3)
    gK = g(z1K, z2K, z3K)
    for i,j in itertools.combinations(range(N), 2):
        if gcd(i ,N) != gcd(j, N):
            if torsion_closure(RK.ideal([gK, z1K-z^i, z2K-z^j])):
                (i1, j1) = (N//gcd(N, i), N//gcd(N, j))
                (i1, j1) = (min(i1,j1), max(i1, j1))
                if (i1, j1) not in pairs:
                    pairs.append((i1, j1))
print(sorted(pairs))

In [0]:
print("Total time: {} seconds".format(time.time()-t))